In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [4]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_zhishi_kexue=pd.DataFrame(columns=columns)
channel_zhishi_kexue

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [5]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [6]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [14]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[7]/div/a/span').click()  #选择“知识”频道

In [7]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[7]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'知识'

In [8]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').click()  #选择“科学科普”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').text
sub_channel

'科学科普'

In [10]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [11]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_zhishi_kexue.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-21 07:49:26.427278


<ipython-input-11-3faa76596dfa>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-3faa76596dfa>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-3faa76596dfa>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1aJ411j7n7视频属性的抓取
BV号为1aJ411j7n7的视频属性抓取结束


<ipython-input-11-3faa76596dfa>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1n7411e7Fm视频属性的抓取
BV号为1n7411e7Fm的视频属性抓取结束
开始BV号为1wJ411n7zU视频属性的抓取
BV号为1wJ411n7zU的视频属性抓取结束
开始BV号为1pJ411n7FC视频属性的抓取
BV号为1pJ411n7FC的视频属性抓取结束
开始BV号为1S7411q7wp视频属性的抓取
BV号为1S7411q7wp的视频属性抓取结束
开始BV号为177411r7vb视频属性的抓取
BV号为177411r7vb的视频属性抓取结束
开始BV号为1gJ411E7eA视频属性的抓取
BV号为1gJ411E7eA的视频属性抓取结束
开始BV号为127411v7pm视频属性的抓取
BV号为127411v7pm的视频属性抓取结束
开始BV号为1j7411z7KQ视频属性的抓取
BV号为1j7411z7KQ的视频属性抓取结束
开始BV号为1z7411e7qB视频属性的抓取
BV号为1z7411e7qB的视频属性抓取结束
开始BV号为1G7411i7M1视频属性的抓取
BV号为1G7411i7M1的视频属性抓取结束
开始BV号为1MJ411J7bs视频属性的抓取
BV号为1MJ411J7bs的视频属性抓取结束
开始BV号为1AJ411j7vE视频属性的抓取
BV号为1AJ411j7vE的视频属性抓取结束
开始BV号为1v7411z7Jp视频属性的抓取
BV号为1v7411z7Jp的视频属性抓取结束
开始BV号为1rJ411n7Ue视频属性的抓取
BV号为1rJ411n7Ue的视频属性抓取结束
开始BV号为1W7411e7qG视频属性的抓取
BV号为1W7411e7qG的视频属性抓取结束
开始BV号为1L7411i7SC视频属性的抓取
BV号为1L7411i7SC的视频属性抓取结束
开始BV号为1K741167Fw视频属性的抓取
BV号为1K741167Fw的视频属性抓取结束
开始BV号为1eJ411E7Qa视频属性的抓取
BV号为1eJ411E7Qa的视频属性抓取结束
开始BV号为1oJ411E7Lg视频属性的抓取
BV号为1oJ411E7Lg的视频属性抓取结束
开始BV号为1s7411e7H8视频属性的抓取
BV号为1s7411e7H8的视频属性抓取结束
开始BV号为1dJ411j78t视频属性的抓取
BV号为1dJ411j78t的视

...

开始BV号为1dC4y1p71M视频属性的抓取
BV号为1dC4y1p71M的视频属性抓取结束
开始BV号为11i4y1t799视频属性的抓取
BV号为11i4y1t799的视频属性抓取结束
开始BV号为1W54y1d73h视频属性的抓取
BV号为1W54y1d73h的视频属性抓取结束
开始BV号为1WK411L7DP视频属性的抓取
BV号为1WK411L7DP的视频属性抓取结束
开始BV号为1we411s7S2视频属性的抓取
BV号为1we411s7S2的视频属性抓取结束
开始BV号为1jK4y1k7by视频属性的抓取
BV号为1jK4y1k7by的视频属性抓取结束
开始BV号为1vi4y1t77f视频属性的抓取
BV号为1vi4y1t77f的视频属性抓取结束
开始BV号为1Ae41147f9视频属性的抓取
BV号为1Ae41147f9的视频属性抓取结束
开始BV号为1Se411s76S视频属性的抓取
BV号为1Se411s76S的视频属性抓取结束
开始BV号为17A411h7wJ视频属性的抓取
BV号为17A411h7wJ的视频属性抓取结束
开始BV号为1Kc411h7eP视频属性的抓取
BV号为1Kc411h7eP的视频属性抓取结束
开始BV号为1NT4y1V7PH视频属性的抓取
BV号为1NT4y1V7PH的视频属性抓取结束
开始BV号为1Y54y1d7Vc视频属性的抓取
BV号为1Y54y1d7Vc的视频属性抓取结束
开始BV号为1xK41157ow视频属性的抓取
BV号为1xK41157ow的视频属性抓取结束
开始BV号为1aT4y157a1视频属性的抓取
BV号为1aT4y157a1的视频属性抓取结束
开始BV号为1H54y1d7QN视频属性的抓取
BV号为1H54y1d7QN的视频属性抓取结束
开始BV号为15K4y1b7gg视频属性的抓取
BV号为15K4y1b7gg的视频属性抓取结束
开始BV号为1Df4y1S7ek视频属性的抓取
BV号为1Df4y1S7ek的视频属性抓取结束
开始BV号为13p4y1y7wr视频属性的抓取
BV号为13p4y1y7wr的视频属性抓取结束
开始BV号为16t4y127LD视频属性的抓取
BV号为16t4y127LD的视频属性抓取结束
开始BV号为1UZ4y1j714视频属性的抓取
BV号为1UZ4y1j714的视

开始BV号为1qK4y1x7RE视频属性的抓取
BV号为1qK4y1x7RE的视频属性抓取结束
开始BV号为18z411i7v7视频属性的抓取
BV号为18z411i7v7的视频属性抓取结束
开始BV号为1Pg4y1q7CJ视频属性的抓取
BV号为1Pg4y1q7CJ的视频属性抓取结束
开始BV号为1gZ4y1H7zg视频属性的抓取
BV号为1gZ4y1H7zg的视频属性抓取结束
开始BV号为1UK4y1x736视频属性的抓取
BV号为1UK4y1x736的视频属性抓取结束
开始BV号为1z54y1B72h视频属性的抓取
BV号为1z54y1B72h的视频属性抓取结束
开始BV号为1jz411v7wm视频属性的抓取
BV号为1jz411v7wm的视频属性抓取结束
开始BV号为1xZ4y1H7HR视频属性的抓取
BV号为1xZ4y1H7HR的视频属性抓取结束
开始BV号为1kK4y1x7cP视频属性的抓取
BV号为1kK4y1x7cP的视频属性抓取结束
开始BV号为16A411v7Gd视频属性的抓取
BV号为16A411v7Gd的视频属性抓取结束
开始BV号为1mi4y1s7tD视频属性的抓取
BV号为1mi4y1s7tD的视频属性抓取结束
开始BV号为1MK411p7zB视频属性的抓取
BV号为1MK411p7zB的视频属性抓取结束
开始BV号为1Tz4y1R7VU视频属性的抓取
BV号为1Tz4y1R7VU的视频属性抓取结束
开始BV号为1AC4y1Y73g视频属性的抓取
BV号为1AC4y1Y73g的视频属性抓取结束
开始BV号为1Mv411B7Ga视频属性的抓取
BV号为1Mv411B7Ga的视频属性抓取结束
开始BV号为1fv411q7PK视频属性的抓取
BV号为1fv411q7PK的视频属性抓取结束
开始BV号为1Bz4y197M5视频属性的抓取
BV号为1Bz4y197M5的视频属性抓取结束
开始BV号为1DD4y1D7ZJ视频属性的抓取
BV号为1DD4y1D7ZJ的视频属性抓取结束
开始BV号为1wC4y1Y7LM视频属性的抓取
BV号为1wC4y1Y7LM的视频属性抓取结束
开始BV号为1vz4y197QA视频属性的抓取
BV号为1vz4y197QA的视频属性抓取结束
开始BV号为1UV411r72v视频属性的抓取
BV号为1UV411r72v的视

<ipython-input-11-3faa76596dfa>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1BT4y157Qo视频属性的抓取
BV号为1BT4y157Qo的视频属性抓取结束
开始BV号为1QK4y1Y7Xq视频属性的抓取
BV号为1QK4y1Y7Xq的视频属性抓取结束
开始BV号为1j54y1i7iC视频属性的抓取
BV号为1j54y1i7iC的视频属性抓取结束
开始BV号为1jh411d7cD视频属性的抓取
BV号为1jh411d7cD的视频属性抓取结束
开始BV号为1VC4y1t73T视频属性的抓取
BV号为1VC4y1t73T的视频属性抓取结束
开始BV号为1C54y1U787视频属性的抓取
BV号为1C54y1U787的视频属性抓取结束
开始BV号为12i4y1g7G7视频属性的抓取
BV号为12i4y1g7G7的视频属性抓取结束
开始BV号为1sh411Z7GV视频属性的抓取
BV号为1sh411Z7GV的视频属性抓取结束
开始BV号为1154y1e7Uj视频属性的抓取
BV号为1154y1e7Uj的视频属性抓取结束
开始BV号为1pi4y1g7js视频属性的抓取
BV号为1pi4y1g7js的视频属性抓取结束
开始BV号为13z4y1Q7pU视频属性的抓取
BV号为13z4y1Q7pU的视频属性抓取结束
开始BV号为185411h79H视频属性的抓取
BV号为185411h79H的视频属性抓取结束
开始BV号为1Nt4y1S77p视频属性的抓取
BV号为1Nt4y1S77p的视频属性抓取结束
开始BV号为1fV411z7kw视频属性的抓取
BV号为1fV411z7kw的视频属性抓取结束
开始BV号为1BT4y157At视频属性的抓取
BV号为1BT4y157At的视频属性抓取结束
开始BV号为1NK411M7kS视频属性的抓取
BV号为1NK411M7kS的视频属性抓取结束
开始BV号为1yk4y127Ch视频属性的抓取
BV号为1yk4y127Ch的视频属性抓取结束
开始BV号为1pC4y1t7KN视频属性的抓取
BV号为1pC4y1t7KN的视频属性抓取结束
开始BV号为1rA411Y7fE视频属性的抓取
BV号为1rA411Y7fE的视频属性抓取结束
开始BV号为1z541187ey视频属性的抓取
BV号为1z541187ey的视频属性抓取结束
开始BV号为1xC4y1875J视频属性的抓取
BV号为1xC4y1875J的视

BV号为15p4y1e7HA的视频属性抓取结束
开始BV号为1LK4y1e7Jo视频属性的抓取
BV号为1LK4y1e7Jo的视频属性抓取结束
开始BV号为1Pi4y1E7ZR视频属性的抓取
BV号为1Pi4y1E7ZR的视频属性抓取结束
开始BV号为17K4y1e7pw视频属性的抓取
BV号为17K4y1e7pw的视频属性抓取结束
开始BV号为1oK4y1h7un视频属性的抓取
BV号为1oK4y1h7un的视频属性抓取结束
开始BV号为17T4y1c7gt视频属性的抓取
BV号为17T4y1c7gt的视频属性抓取结束
开始BV号为18z4y1Z7dN视频属性的抓取
BV号为18z4y1Z7dN的视频属性抓取结束
开始BV号为17h41197jR视频属性的抓取
BV号为17h41197jR的视频属性抓取结束
开始BV号为1By4y1y7N7视频属性的抓取
BV号为1By4y1y7N7的视频属性抓取结束
开始BV号为1kk4y1y7GB视频属性的抓取
BV号为1kk4y1y7GB的视频属性抓取结束
开始BV号为1hV411m7fE视频属性的抓取
BV号为1hV411m7fE的视频属性抓取结束
开始BV号为1Pp4y1e73Z视频属性的抓取
BV号为1Pp4y1e73Z的视频属性抓取结束
开始BV号为155411j7ww视频属性的抓取
BV号为155411j7ww的视频属性抓取结束
开始BV号为18K4y1a7r3视频属性的抓取
BV号为18K4y1a7r3的视频属性抓取结束
开始BV号为1DV41127yv视频属性的抓取
BV号为1DV41127yv的视频属性抓取结束
开始BV号为1hv41117M2视频属性的抓取
BV号为1hv41117M2的视频属性抓取结束
开始BV号为1JD4y1o7US视频属性的抓取
BV号为1JD4y1o7US的视频属性抓取结束
开始BV号为1aT4y1A7Lv视频属性的抓取
BV号为1aT4y1A7Lv的视频属性抓取结束
开始BV号为1n5411j7yG视频属性的抓取
BV号为1n5411j7yG的视频属性抓取结束
开始BV号为1jK411K73j视频属性的抓取
BV号为1jK411K73j的视频属性抓取结束
开始BV号为1zt4y1q7Ea视频属性的抓取
BV号为1zt4y1q7Ea的视频属性抓取结束
开始BV号为1ST4y1L7av

In [29]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-29-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [12]:
channel_zhishi_kexue.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
5月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
6月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [13]:
channel_zhishi_kexue.to_json('channel_zhishi_kexue.json')

In [3]:
channel_zhishi_kexue=pd.read_json('channel_zhishi_kexue.json')

In [4]:
pd.set_option('display.max_rows', None)

In [9]:
channel_zhishi_kexue[channel_zhishi_kexue['时间']=='5月']

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1Cg4y1q7nr,5月,龙女之声,个人认证,117.1万,95,不要什么都往DNA里刻啊！【龙女动物学12 】腹足纲,知识,科学科普,活动作品,自制,"[生物, 科普, 知识, 科学科普, 我是行走的博物馆, 动物, 腹足纲, 软体动物, 海兔...",10:31,126.3万,13.4万,8.5万,2.2万,2788,NaN,6484,否
1Rk4y167zm,5月,另类视角,无,1911,10,三维世界在二维世界的投影,知识,科学科普,普通视频,转载,"[宇宙, 科学, 科普, 知识, 科学科普, 2D, 实验, 三维, 3D, 二维]",00:40,124.5万,5.3万,2706,1.5万,1891,1276.0,269,否
1kA411t76g,5月,回形针PaperClip,个人认证,276.6万,174,【回形针PaperClip】开锁师傅对你的锁做了什么？,知识,科学科普,普通视频,自制,"[冷知识, 科学, 科普, 知识, 科学科普, 趣闻, 回形针, 硬核, 制造业, 开锁, 物理]",05:17,116.5万,5.3万,1.9万,1.2万,6228,NaN,5041,否
1754y1X72Y,5月,无穷小亮的科普日常,个人认证,127.5万,120,【亮记生物鉴定】近期网络热传生物鉴定,知识,科学科普,活动作品,自制,"[生物, 人文, 科普, 知识, 科学科普, 自然, 植物, 动物, 知识分享官]",04:10,114.6万,6.5万,9540,5504,2520,NaN,4516,否
1Qa4y1v77S,5月,科技狐,个人认证,32.2万,348,【科普】016期-你思考的时候，脑中的声音到底是谁？,知识,科学科普,活动作品,自制,"[生物, 健康, 科学, 科普, 知识, 科学科普, 知识分享官, 基因, 神经]",04:34,113.1万,6.6万,6243,2.0万,2830,7762.0,1.2万,否
19K4y1b7JF,5月,兔叭咯,个人认证,345.9万,188,医学博士：那啥会起痘痘吗？I 医美是骗人的吗？I 烂脸怎么办？,知识,科学科普,活动作品,自制,"[知识, 科学科普, 学习, 时尚新势力-时尚新秀召集令, 护肤品, 祛痘, 校园, 医学,...",07:47,110.4万,7.7万,3.1万,1.7万,7721,NaN,4824,否
1ff4y117db,5月,极客队长GeekLead,个人认证,37.7万,42,华为崛起，都靠中国人吗？【科普】改写历史的技术突破,知识,科学科普,活动作品,自制,"[5G, iPhone, 人工智能, 科学, 知识, 科学科普, 数码, 华为, 知识分享官...",11:53,106.0万,9.3万,6.9万,3.0万,1.0万,NaN,2.8万,否
1jK411s7hm,5月,回形针PaperClip,个人认证,276.6万,174,【回形针PaperClip】ICU 病房里究竟发生了什么？,知识,科学科普,普通视频,自制,"[科学, 知识, 科学科普, 回形针, ICU, 知识, 硬核, 医学设备]",05:02,104.4万,5.0万,2.0万,7515,3184,NaN,4770,否
1V5411x7Pi,5月,RealChubbyemu,个人认证,39.1万,38,一个学生三小时内喝了两加仑咖啡，这是他肾脏发生的变化,知识,科学科普,活动作品,自制,"[咖啡, 科普, 知识, 科学科普, Chubbyemu, 涨姿势, 医学, 知识分享官, ...",13:36,102.6万,2.3万,6306,1.1万,9652,NaN,5565,是
1Rt4y117v6,5月,工程师徐小刀,个人认证,10.6万,33,三峡舍弃大量发电效益，水位降至145米，才是体现它真正的价值！,知识,科学科普,普通视频,自制,"[武汉, 知识, 科学科普, 三峡大坝, 三峡, 分洪区, 三峡水电站, 荆江, 洪水, 长...",04:49,101.4万,6.1万,1.4万,6510,1699,NaN,2551,否
